In [1]:
pip install music21

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:

import numpy as np

In [4]:
from music21 import *

In [5]:
import pandas as pd

In [6]:
s = corpus.parse('bach/bwv65.2.xml')

In [7]:
s.analyze('key')

<music21.key.Key of a minor>

In [8]:
from music21 import corpus, meter, tempo, expressions

In [9]:
configure.run()



______________________________________________________________________________ 
  
Welcome to the music21 Configuration Assistant. You will be guided through a 
number of questions to install and set up music21. Simply pressing return at a 
prompt will select a default, if available. 

You may run this configuration again at a later time by running 
music21/configure.py. 
  
______________________________________________________________________________ 
  
Defining an XML Reader permits automatically opening music21-generated 
MusicXML in an editor for display and manipulation when calling the show() 
method. Setting this option is highly recommended. 
  
[1] /Applications/MuseScore 3.app 
  
Choose a number from the preceding options (default is 1): 
MusicXML Reader set to: /Applications/MuseScore 3.app 
  
______________________________________________________________________________ 
  
The BSD-licensed music21 software is distributed with a corpus of encoded 
compositions which ar

In [10]:
nocturne=converter.parse('/Users/melodyqian/Documents/GitHub/pianocompetitionpredictions/test/chopnoc9.mxl')

In [11]:
#First: number of Notes This is is more coomplicated than the chord function because I had to count each note in the chord.
nocflat=nocturne.flat

number_of_notes = 0
for elem in nocflat:
    if isinstance(elem, note.Note):
        number_of_notes += 1
    elif isinstance(elem, chord.Chord):
        number_of_notes += len(elem.pitches)

print(number_of_notes)

1253


In [12]:
#Number of chords
nocchords = [element for element in nocflat if isinstance(element, chord.Chord)]

print(len(nocchords))



301


In [13]:
#Pitchwise distances between chords, we go by the root of the chord or the lowest note.
def chordpitchdistances(chords):
    pitchdistances= []
    for i in range(len(chords)-1):
        firstchord= chords[i]
        secondchord= chords[i+1]
        rootInterval=interval.Interval(firstchord.root(),secondchord.root())
        pitchdistances.append(rootInterval.semitones)
    return(pitchdistances)

nocpitchdistances = chordpitchdistances(nocchords)
noc_abs_dis= [abs(dis) for dis in nocpitchdistances]
#print(noc_abs_dis)
print(np.mean(noc_abs_dis))


7.006666666666667


In [14]:
def chordtimedistances(chords):
    timedistances=[]
    for i in range(len(chords)-1):
        firstchord= chords[i]
        secondchord= chords[i+1]
        time_difference = secondchord.offset - firstchord.offset
        timedistances.append(time_difference)  # Add time difference to the list
    return timedistances

noctimedistances = chordtimedistances(nocchords)
print(np.mean(noctimedistances))




0.695


In [15]:
print(nocflat.analyze('key'))
noctimesignatures=nocflat.getElementsByClass(meter.TimeSignature)
print(len(noctimesignatures))
#I will put these in here for validation purposes to make sure our database doesn't accidentally find the wrong piece
print(nocturne.metadata.title)
print(nocturne.metadata.composer)


E- major
8
Nocturne  Op.9  No.2
Fredrik Chopin


In [16]:
def findtrills(score):
    trills=0
    totalbeats=0
    for element in score.notesAndRests:
        if element.expressions:  # Check if the note has expressions
            for expr in element.expressions:
                    if isinstance(expr, expressions.Trill):  # Check if it's a trill
                        trills+= 1 
                        totalbeats+=element.quarterLength
    return [trills, totalbeats]

print(findtrills(nocflat))

[2, 0.5]


In [17]:
for element in nocflat.notes:
    if element.expressions:
        print(element, element.expressions)

<music21.note.Note C> [<music21.expressions.Turn>]
<music21.note.Note F> [<music21.expressions.InvertedMordent>]
<music21.note.Note E-> [<music21.expressions.InvertedMordent>]
<music21.note.Note E-> [<music21.expressions.InvertedMordent>]
<music21.note.Note E-> [<music21.expressions.Trill>]
<music21.note.Note E-> [<music21.expressions.Turn>]
<music21.note.Note E-> [<music21.expressions.InvertedMordent>]
<music21.note.Note A-> [<music21.expressions.Trill>]
<music21.chord.Chord B-5 B-6> [<music21.expressions.Fermata>]
<music21.note.Note D> [<music21.expressions.Fermata>]
<music21.chord.Chord G3 E-4> [<music21.expressions.Fermata>]
<music21.chord.Chord E-2 B-2> [<music21.expressions.Fermata>]


In [18]:
noctempo= nocflat.getElementsByClass(tempo.MetronomeMark)
nocBPMdict = {}
for tm in noctempo:
    bpm=tm.getQuarterBPM()
    measureNumber=tm.measureNumber
    nocBPMdict[bpm]=measureNumber
print(nocBPMdict)

#key is bpm, value is measure number. now let's take a weighted average.
weighted_tempo=[]
sorted_tempos = sorted(noctempo, key=lambda tm: tm.measureNumber)
for i in range(len(sorted_tempos)):
    current_bpm = sorted_tempos[i].getQuarterBPM()
    current_measure = sorted_tempos[i].measureNumber
    if i < len(sorted_tempos) - 1:
        next_measure = sorted_tempos[i + 1].measureNumber
        duration = next_measure - current_measure
    else:
        # If this is the last tempo, assume it lasts to the end of the piece
        duration = nocturne.highestTime
    weighted_tempo.append((current_bpm, duration))


total_weight = sum(duration for _, duration in weighted_tempo)
weighted_mean = sum(bpm * duration for bpm, duration in weighted_tempo) / total_weight 
print(f"{weighted_mean:.2f}")
    

{60.0: 36, 35.0: 35, 40.0: 33, 70.0: 33, 80.0: 34, 50.0: 35, 20.0: 35}
59.96


In [ ]:
#number of measures
num_measures = len(nocturne.recurse().getElementsByClass('Measure'))
print(num_measures/2)
#need to divide measures by 2 because it counts the left and right hand parts separately.

38.0


In [21]:
from collections import Counter

duration_counter = Counter()

for note in nocturne.recurse().notes:
    duration_counter[note.duration.quarterLength] += 1

num_32nd_notes_and_shorter = sum(count for duration, count in duration_counter.items() if duration <= 0.125)
print(num_32nd_notes_and_shorter)

num_16_notes=sum(count for duration, count in duration_counter.items() if duration==.25)
print(num_16_notes)

num_eighth=sum(count for duration, count in duration_counter.items() if duration==0.5)
print(num_eighth)

num_quarter=sum(count for duration, count in duration_counter.items() if duration==1)
print(num_quarter)

num_half=sum(count for duration, count in duration_counter.items() if duration==2)
print(num_half)

sample= [106, 122, 494, 20, 0]

106
122
494
20
0


In [22]:
# Define note durations
note_durations = {
    '32nd_and_shorter': 0.125,
    '16th': 0.25,
    'eighth': 0.5,
    'quarter': 1,
    'dotted quarter': 1.5,
    'half': 2,
    'dotted half': 3,
    'whole': 4
}

duration_counter = Counter(note.duration.quarterLength for note in nocturne.recurse().notes)

def count_notes(nocturne, duration_counter):
    note_counts= {}
    for name, duration in note_durations.items():
        if name=='32nd_and_shorter':
            note_counts[name]= sum(count for d, count in duration_counter.items() if d<=duration)
        else:
            note_counts[name]=duration_counter[duration]
    return note_counts

note_counts = count_notes(nocturne, duration_counter)

for name, count in note_counts.items():
    print(f"{name}: {count}")

total_notes = sum(note_counts.values())
print(total_notes)

32nd_and_shorter: 106
16th: 122
eighth: 494
quarter: 20
dotted quarter: 48
half: 0
dotted half: 9
whole: 0
799


In [25]:
count_notes(nocturne,duration_counter)

{'32nd_and_shorter': 106,
 '16th': 122,
 'eighth': 494,
 'quarter': 20,
 'dotted quarter': 48,
 'half': 0,
 'dotted half': 9,
 'whole': 0}

In [24]:
note_counts_list = []

for i, row in difficultymetric.iterrows():
    counts = count_notes(parse(row['filepath']), get_durations(row['filepath']))
    note_counts_list.append(counts)

note_counts_df = pd.DataFrame(note_counts_list)

NameError: name 'difficultymetric' is not defined